# Construction

### Whole genome

Make an output folder

In [ ]:
!mkdir output

Construct a graph from the yeast data with a maximum node sequence size of 1000 

In [4]:
!vg construct -r data/SGD_2010.fasta \
    -v data/SGRP2-cerevisiae-freebayes-snps-Q30-GQ30.vcf.gz \
    -m 1000 -a > output/yeastgraph.vg


Investigate the graphs statistics, this graph has 714603 nodes and 969760 edges.

In [5]:
!vg stats -z output/yeastgraph.vg

nodes	714603
edges	969760


### Each chromosome

Make a chromosomes folder

In [15]:
!mkdir output/chromosomes

Get chromosome names from the VCF file

In [9]:
!zgrep -v "#" data/SGRP2-cerevisiae-freebayes-snps-Q30-GQ30.vcf.gz \
| cut -f1 | uniq > chromosomes.txt

Construct a graph for each chromosome with a maximum node sequence size of 1000

In [16]:
!for chrom in $(cat chromosomes.txt); \
    do vg construct -r data/SGD_2010.fasta \
    -v data/SGRP2-cerevisiae-freebayes-snps-Q30-GQ30.vcf.gz \
    -m 1000 -R $chrom -a > output/chromosomes/yeastgraph${chrom}.vg; \
done


Show statistics for first five chromosome graphs

In [19]:
!for chrom in $(cat chromosomes.txt | head -5); \
    do echo $chrom: ; vg stats -z output/chromosomes/yeastgraph$chrom.vg; \
done


chr1:
nodes	24288
edges	33786
chr2:
nodes	46685
edges	63193
chr3:
nodes	21108
edges	28659
chr4:
nodes	86116
edges	116698
chr5:
nodes	30962
edges	41874


# Pruning


Unable to immediately index the graph, and unable to prune the entire graph. Therefore need to prune chromosome by chromosome


In [ ]:
#Doesn't complete
!vg mod -pl 16 -e 3 -t 4 output/yeastgraph.vg \ 
    | vg mod -S -l 32 -t 4 - \
    > output/yeastgraphpruned.vg;


Prune each chromosome

In [20]:
!for chrom in $(cat chromosomes.txt); \
    do vg mod -pl 16 -e 3 -t 16 \
    output/chromosomes/yeastgraph${chrom}.vg \
    | vg mod -S -l 32  - \
    > output/chromosomes/yeastgraph${chrom}pruned.vg; \
done

Notice the result of pruning

In [21]:
!for chrom in $(cat chromosomes.txt | head -5); \
    do echo $chrom; \
    vg stats -z output/chromosomes/yeastgraph${chrom}pruned.vg; \
done


chr1
nodes	15751
edges	20970
chr2
nodes	43218
edges	57993
chr3
nodes	19296
edges	25864
chr4
nodes	78802
edges	105634
chr5
nodes	28944
edges	38813


Now index each chromosome

In [22]:
!for chrom in $(cat chromosomes.txt); \
    do   echo $chrom; \
         vg index -x output/chromosomes/yeastgraph${chrom}pruned.xg \
         -g output/chromosomes/yeastgraph${chrom}pruned.gcsa \
         -k 16 \
         -t 4 \
         output/chromosomes/yeastgraph${chrom}pruned.vg; \
done


chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16


Put chromosome data in different chromosome files

In [ ]:
!for chrom in $(cat chromosomes.txt); \
    do mkdir output/chromosomes/$chrom; \
    mv output/chromosomes/yeastgraph${chrom}pruned.xg \
        output/chromosomes/$chrom; \
    mv output/chromosomes/yeastgraph${chrom}pruned.gcsa \
        output/chromosomes/$chrom; \
    mv output/chromosomes/yeastgraph${chrom}pruned.vg \
        output/chromosomes/$chrom; \
    mv output/chromosomes/yeastgraph${chrom}.vg \
        output/chromosomes/$chrom; \
done

Simulate 1 read of length 20 with and error rate of 0.05 and an indel frequency rate of 0.02

In [158]:
!vg sim -l 20 -n 1 -e 0.05 -i 0.02 \
    output/chromosomes/chr1/yeastgraphchr1.vg \
    > output/chromosomes/chr1/simulate.reads

In [159]:
!cat output/chromosomes/chr1/simulate.reads

TACAGCAGATTCGAGTTTTT


Map these reads to the graph

In [160]:
!vg map -r output/chromosomes/chr1/simulate.reads \
    -x output/chromosomes/chr1/yeastgraphchr1pruned.xg \
    -g output/chromosomes/chr1/yeastgraphchr1pruned.gcsa \
    -k 22 \
    > output/chromosomes/chr1/mappedreads.gam


View the alignment information

In [172]:
!vg view -a output/chromosomes/chr1/mappedreads.gam

{"sequence": "TACAGCAGATTCGAGTTTTT", "path": {"mapping": [{"position": {"node_id": 6912}, "edit": [{"from_length": 1, "to_length": 1}], "rank": 1}, {"position": {"node_id": 6914}, "edit": [{"from_length": 8, "to_length": 8}], "rank": 2}, {"position": {"node_id": 6916}, "edit": [{"from_length": 1, "to_length": 1}], "rank": 3}, {"position": {"node_id": 6917}, "edit": [{"from_length": 10, "to_length": 10}], "rank": 4}]}, "score": 40}


Find the alignment node and display the alignment.

In [173]:
!vg find -n 6915 -c 5 \
    -x output/chromosomes/chr1/yeastgraphchr1pruned.xg \
    -g output/chromosomes/chr1/yeastgraphchr1pruned.gcsa \
    | vg view -d -A output/chromosomes/chr1/mappedreads.gam - | dot -Tsvg > test.svg


In [170]:
%%HTML
test.svg
